# Setup

In [1]:
#_____PACKAGE DEPENDENCIES_____
import cvxpy as cp
import numpy as np

In [31]:
'''
# the functions for calculating the terms in the optimization function
# written in python to be more readable (no list comprehension)

def dist(schedule,P,N):
    dist = 0
    for i in range(N):
        for j in range(N):
            if P[i][j] > 0.99:
                dist += schedule[j].index(1) - schedule[i].index(1)
    return(dist)

def deviation(schedule, L, S, N, A):
    diff = 0
    for s in range(S):
        ideal = A[s]/ sum(A)
        load = sum([schedule[c][s]*L[c]/sum(L) for c in range(N)])
        diff += abs(load-ideal)
    return(diff)
''';

# Optization Problem and Solution

In [103]:
#_____PRINTS SCHEDULE BASED ON DECISION VAIRABLE RESULTS_____

def recoverSchedule(matrix):
    if matrix is None:
        print("No Solution")
    else:
        matrix = [[v > 0.99 for v in row] for row in matrix]
        print("________")
        c = len(matrix)
        s = len(matrix[0])
        print("Schedule:")
        for si in range(s):
            courses = []
            for ci in range(c):
                if matrix[ci][si]:
                    courses.append(ci)
            course_str = ",".join([str(c+1) for c in courses])
            print("interval {}: {}".format(str(si+1),course_str))
        print("________")
    return(0)

In [104]:
#_____OPTIMIZATION FUNCTION_____

def calcSchedule(N,S,phi,L,P,A):
    #_____DECISION VARIABLES_____
    #x[i][j] is boolean if course i assigned in semester j
    x = cp.Variable((N,S),boolean=True)

    #_____OPTIMIZATION FUNCTION_____
    # distance between course and prereq
    dist = cp.sum([(x[j]*np.matrix([s for s in range(S)]).T)[0] - (x[i]*np.matrix([s for s in range(S)]).T)[0] for i in range(N) for j in range(N) if P[i][j] > 0.99])
    
    # deviation from ideal course load
    ## course load for that month is adjusted to a proportion of total course load
    ## ideal course load for that month is the proportion of availability for that month
    ## to the sum of of availability A[s]/sum(A)
    dev = sum(sum(cp.abs((x[c][s]*L[c]/sum(L)) - A[s]/sum(A)) for c in range(N)) for s in range(S))

    f = phi*dist + (1-phi)*dev

    #_____CONSTRAINTS_____
    g = []

    # all courses must be allocated to exactly 1 period
    g.extend([cp.sum(x,axis=1) == np.ones(N)])
    # all prereqs must come first
    for i in range(N):
        for j in range(N):
            if P[i][j] > 0.99:
                g.extend([(x[j]*np.matrix([s for s in range(S)]).T)[0] >= (x[i]*np.matrix([s for s in range(S)]).T)[0] + 1])
    '''
    Extension:
    All course loads for a segment must be below max availability

    sum(x[i][sem] * load[i]) <= A[sem]
    '''
    for t in range(S):
        g.extend([sum(cp.multiply(x[:,t], L)) <= A[t]])


    sol = cp.Problem(cp.Minimize(f), g)
    print("Solution:", sol.solve())
    if type(dist) == int:
        print("dist: ", dist)
    else:
        print("dist: ", dist.value)
    if type(dev) == int:
        print("dev: ", dev)
    else:
        print("dev: ", dev.value)
    # ____ CONVERT TO TRUE FALSE _____
    recoverSchedule(x.value)
    return(0)

In [105]:
#_____TRIAL_____

N = 4 # num courses
S = 3 # num semesters
phi = 0.5
L = np.array([2,4,2,4]) # load by course
# P[i][j] = 1 iff i is a prereq of j
P = np.array([[0, 0, 0, 0], 
          [1, 0, 0, 0],
          [0, 0, 0, 0],
          [1, 0, 1, 0]]) 

#course loads/ time availabilities for independent learners
A = np.array([8,4,0]) # availability by month

calcSchedule(N,S,phi,L,P,A);

Solution: 3.000000000131738
dist:  2.9999999999287885
dev:  3.000000000018779
________
Schedule:
interval 1: 2,4
interval 2: 1,3
interval 3: 
________


# Model Response to Parameters

In [115]:
#_____Prioritization_____

N = 4
S = 2
pphi = np.linspace(0,1,3)

P = np.array([[0, 0, 0, 0], 
          [1, 0, 0, 0],
          [0, 0, 0, 0],
          [1, 0, 0, 0]]) 

A = np.array([20,20]) # availability by month

L = np.array([5,5,5,5]) # small values

for phi in pphi:
    print("***********************************")
    calcSchedule(N,S,phi,L,P,A);

***********************************
Solution: 2.9999999987041854
dist:  2.000000000055042
dev:  2.9999999999999796
________
Schedule:
interval 1: 2,4
interval 2: 1,3
________
***********************************
Solution: 2.499999999988322
dist:  2.0000000000015183
dev:  3.0000000000000018
________
Schedule:
interval 1: 2,3,4
interval 2: 1
________
***********************************
Solution: 1.9999999999910176
dist:  1.999999999991018
dev:  3.000000000000009
________
Schedule:
interval 1: 2,4
interval 2: 1,3
________


In [107]:
#_____Course Load_____

N = 4
S = 2
phi = 0.5

P = np.array([[0, 0, 0, 0], 
          [1, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 1, 0]]) 

A = np.array([20,20]) # availability by month

L1 = np.array([1,1,1,1]) # small values
L2 = np.array([10,10,10,10]) # medium values
L3 = np.array([1,10,10,1]) # random mixture of values
L4 = np.array([1,10,1,10]) # prereqs are a lot smaller
L5 = np.array([100,100,100,100]) # large values
L6 = np.array([5,5,5,5]) # large but feasible values

pL = [L1,L2,L3,L4,L5,L6]

for L in pL:
    print("***********************************")
    calcSchedule(N,S,phi,L,P,A);

***********************************
Solution: 2.5000000000041687
dist:  1.9999999999972813
dev:  3.0000000000000018
________
Schedule:
interval 1: 2,4
interval 2: 1,3
________
***********************************
Solution: 2.499999999991916
dist:  2.00000000000708
dev:  3.000000000000002
________
Schedule:
interval 1: 2,4
interval 2: 1,3
________
***********************************
Solution: 2.50000000000932
dist:  1.9999999999973297
dev:  2.9999999999999996
________
Schedule:
interval 1: 2,4
interval 2: 1,3
________
***********************************
Solution: 2.5000000000726454
dist:  2.0000000000046607
dev:  2.9999999999998743
________
Schedule:
interval 1: 2,4
interval 2: 1,3
________
***********************************
Solution: None
dist:  None
dev:  None
No Solution
***********************************
Solution: 2.499999999991602
dist:  2.000000000000861
dev:  3.0
________
Schedule:
interval 1: 2,4
interval 2: 1,3
________


In [109]:
#_____Course Dependencies_____

N = 4
S = 2
phi = 0.5

P1 = np.array([[0, 0, 0, 0], 
          [1, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 1, 0]]) 

P2 = np.array([[0, 0, 0, 0], 
          [1, 0, 0, 0],
          [0, 0, 0, 0],
          [1, 0, 1, 0]]) 

P3 = np.array([[0, 0, 0, 0], 
          [1, 0, 0, 0],
          [1, 0, 0, 0],
          [1, 0, 1, 0]]) 

A = np.array([20,20]) # availability by month

L = np.array([10,10,10,10]) # medium values

pP = [P1,P2,P3]

for P in pP:
    print("***********************************")
    calcSchedule(N,S,phi,L,P,A);

***********************************
Solution: 2.499999999991916
dist:  2.00000000000708
dev:  3.000000000000002
________
Schedule:
interval 1: 2,4
interval 2: 1,3
________
***********************************
Solution: 2.999999999990302
dist:  3.000000000002443
dev:  3.000000000000002
________
Schedule:
interval 1: 2,4
interval 2: 1,3
________
***********************************
Solution: None
dist:  None
dev:  None
No Solution


In [111]:
#_____Course Dependencies_____

N = 4
S = 2
phi = 0.5

P = np.array([[0, 0, 0, 0], 
          [1, 0, 0, 0],
          [0, 0, 0, 0],
          [1, 0, 0, 0]]) 

A1 = np.array([10,10])
A2 = np.array([20,20])
A3 = np.array([40,40])
A4 = np.array([50,10])

L = np.array([10,10,10,10]) # medium values

pA = [A1,A2,A3,A4]

for A in pA:
    print("***********************************")
    calcSchedule(N,S,phi,L,P,A);

***********************************
Solution: None
dist:  None
dev:  None
No Solution
***********************************
Solution: 2.5000000000034324
dist:  2.0000000000040807
dev:  3.0000000000000036
________
Schedule:
interval 1: 2,4
interval 2: 1,3
________
***********************************
Solution: 2.4999999999877773
dist:  2.000000000001117
dev:  3.000000000000002
________
Schedule:
interval 1: 2,3,4
interval 2: 1
________
***********************************
Solution: 2.5833333333457764
dist:  1.9999999999943538
dev:  3.1666666666645145
________
Schedule:
interval 1: 2,3,4
interval 2: 1
________
